In [1]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16 #Import all the necessary modules
import matplotlib.pyplot as plot

In [2]:
from glob import glob
IMAGESHAPE = [224, 224, 3] #Provide image size as 224 x 224 this is a fixed-size for VGG16 architecture
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)
#3 signifies that we are working with RGB type of images.
training_data = 'chest_xray/train'
testing_data = 'chest_xray/test' #Give our training and testing path

In [3]:
for each_layer in vgg_model.layers:
	each_layer.trainable = False #Set the trainable as False, So that all the layers would not be trained.
classes = glob('chest_xray/train/*') #Finding how many classes present in our train dataset.
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)
final_model = Model(inputs=vgg_model.input, outputs=prediction) #Combine the VGG output and prediction , this all together will create a model.
final_model.summary() #Displaying the summary
final_model.compile( #Compiling our model using adam optimizer and optimization metric as accuracy.
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [4]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255, #importing our dataset to keras using ImageDataGenerator in keras.
								shear_range = 0.2,
								zoom_range = 0.2,
								horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)
training_set = train_datagen.flow_from_directory('chest_xray/train', #inserting the images.
												target_size = (224, 224),
												batch_size = 4,
												class_mode = 'categorical')
test_set = testing_datagen.flow_from_directory('chest_xray/test',
											target_size = (224, 224),
											batch_size = 4,
											class_mode = 'categorical')
fitted_model = final_model.fit_generator( #Fitting the model.
training_set,
validation_data=test_set,
epochs=5,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


C:\Users\HP\AppData\Local\Temp\ipykernel_12576\2470935531.py:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fitted_model = final_model.fit_generator( #Fitting the model.


Epoch 1/5
1304/1304 [==============================] - 774s 593ms/step - loss: 0.2824 - accuracy: 0.9189 - val_loss: 0.3783 - val_accuracy: 0.9151
Epoch 2/5
1304/1304 [==============================] - 655s 503ms/step - loss: 0.2802 - accuracy: 0.9413 - val_loss: 0.4851 - val_accuracy: 0.9054
Epoch 3/5
1304/1304 [==============================] - 657s 503ms/step - loss: 0.2127 - accuracy: 0.9553 - val_loss: 1.5685 - val_accuracy: 0.8269
Epoch 4/5
1304/1304 [==============================] - 627s 481ms/step - loss: 0.2676 - accuracy: 0.9542 - val_loss: 0.6412 - val_accuracy: 0.9071
Epoch 5/5
1304/1304 [==============================] - 653s 500ms/step - loss: 0.2332 - accuracy: 0.9615 - val_loss: 0.7508 - val_accuracy: 0.9087


In [8]:
# plot.plot(fitted_model.history['loss'], label='training loss') #Plotting the accuracies
# plot.plot(fitted_model.history['val_loss'], label='validation loss')
# plot.legend()
# plot.show()
# plot.savefig('LossVal_loss')
# plot.plot(fitted_model.history['acc'], label='training accuracy')
# plot.plot(fitted_model.history['val_acc'], label='validation accuracy')
# plot.legend()
# plot.show()
# plot.savefig('AccVal_acc')
final_model.save('our_model.h5') #Saving the model file.


In [5]:
#  testing the model

In [2]:
from keras_preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import numpy as np

In [4]:
model=load_model('our_model.h5') #Loading our model

In [12]:
to_test_image = image.load_img('C:/Users/HP/OneDrive/Desktop/minor2/chest_xray/val/NORMAL/NORMAL2-IM-1431-0001.jpeg', target_size=(224, 224)) #Loading the image to test

In [14]:
print(to_test_image.size) 

(224, 224)


In [21]:
imagee=image.img_to_array(to_test_image) #Converting the X-Ray into pixels
imagee=np.expand_dims(imagee, axis=0)
img_data=preprocess_input(imagee)

In [22]:
print(img_data.shape)

(1, 224, 224, 3)


In [24]:
#  predicton of the model
prediction=model.predict(img_data)
output = ""
if prediction[0][0]>prediction[0][1]: #Printing the prediction of model.
	print('Person is safe.')
	output = 'Person is safe.'
else:
	print('Person is affected with Pneumonia.')
	output = 'Person is affected with Pneumonia.'
print(f'Predictions: {prediction}')


1/1 [==============================] - 0s 147ms/step
Person is safe.
Predictions: [[1. 0.]]


In [25]:
output

'Person is safe.'